In [ ]:
# Imports, always include this at the beginning

# No science without NumPy
import numpy as np

# Matplotlib for plotting
import matplotlib.pyplot as plt
%matplotlib inline

# for pretty printing the model
from IPython.display import display, Markdown

# Of course we want Glotaran
import glotaran as gta

In [ ]:
# Load the data
dataset = gta.io.TimeExplicitFile("data.ascii").read("dataset1")
data = dataset.get()
time_axis = dataset.get_time_axis()
spectral_axis = dataset.get_spectral_axis()


In [ ]:
# Plot the Data

plt.figure(figsize=(20, 20))

# Plot some time traces
plt.subplot(4, 4, 1)
wl = [700, 690]
plt.title(f'Traces at {wl[0]}, {wl[1]}')
for w in wl:    
    plt.plot(time_axis, data[np.where(spectral_axis > w)[0][0], :])

# Full Data
plt.subplot(4, 4, 2)
plt.title('Data')
plt.pcolormesh(time_axis, spectral_axis, data)

# Plot the spectrum at time=-83
time_zero = -83
time_zero_idx = (np.abs(time_axis-time_zero)).argmin()
plt.subplot(4, 4, 3)
plt.title('Spectrum at t=-83ps')
plt.plot(spectral_axis, data[:, time_zero_idx])


plt.figure(figsize=(20, 20))

lsvd, svals, rsvd = dataset.svd()

# Plot left singular vectors (LSV, times, first 4)
plt.subplot(4, 4, 1)
plt.title('LSV Data')
plt.plot(time_axis, lsvd[:, :4])

# Plot singular values (SV)
plt.subplot(4, 4, 2)
plt.title('SVals Data')
n_svals = min(10, min(len(time_axis), len(spectral_axis)))
plt.plot(range(n_svals), svals[:n_svals], 'ro')
plt.yscale('log')

# Plot right singular vectors (RSV, wavelengths, first 4)
plt.subplot(4, 4, 3)
plt.title('RSV Data')
plt.plot(spectral_axis, rsvd[:, :4])

In [125]:
# Then we can create our model

model_spec = '''
type: kinetic
parameters:   
  - jvec:
    - [1, {vary: false}]
    - [0, {vary: false}]
    - [0, {vary: false}]
    - [0, {vary: false}]
  - irf:
    - ["center", -83.0]
    - ["width", 1.5]
    - [13200.0, "backsweep", {vary: false}]
  - kinetic:
    - 0.2
    - 0.07
    - 0.02
    - 0.00016
      
compartments: [s1, s2, s3, s4]

megacomplexes:
    - label: mc1
      k_matrices: [km1]
      
k_matrices:
  - label: "km1"
    matrix: {
      '("s2","s1")': kinetic.1,
      '("s3","s2")': kinetic.2,
      '("s4","s3")': kinetic.3,
      '("s4","s4")': kinetic.4
    }
    
irf:
  - label: irf
    type: gaussian
    center: irf.center
    width: irf.width
    backsweep: True
    backsweep_period: irf.backsweep

initial_concentrations: 
  - label: init
    parameter: [jvec.0,jvec.1,jvec.2,jvec.3]
    
datasets:
  - label: dataset1
    type: spectral
    megacomplexes: [mc1]
    path: ''
    irf: irf
    initial_concentration: inputD
'''

model = gta.parse(model_spec)
model.set_dataset("dataset1", dataset)
# print(model)
display(Markdown(str(model)))

# Model

_Type_: Kinetic

## Parameter
  * __jvec__:
    * __1__: _Value_: 1.0, _Min_: -inf, _Max_: inf, _Vary_: False, _Fit_: True
    * __2__: _Value_: 0.0, _Min_: -inf, _Max_: inf, _Vary_: False, _Fit_: True
    * __3__: _Value_: 0.0, _Min_: -inf, _Max_: inf, _Vary_: False, _Fit_: True
    * __4__: _Value_: 0.0, _Min_: -inf, _Max_: inf, _Vary_: False, _Fit_: True
  * __irf__:
    * __center__: _Value_: -83.0, _Min_: -inf, _Max_: inf, _Vary_: True, _Fit_: True
    * __width__: _Value_: 1.5, _Min_: -inf, _Max_: inf, _Vary_: True, _Fit_: True
    * __backsweep__: _Value_: 13200.0, _Min_: -inf, _Max_: inf, _Vary_: False, _Fit_: True
  * __kinetic__:
    * __1__: _Value_: 0.2, _Min_: -inf, _Max_: inf, _Vary_: True, _Fit_: True
    * __2__: _Value_: 0.07, _Min_: -inf, _Max_: inf, _Vary_: True, _Fit_: True
    * __3__: _Value_: 0.02, _Min_: -inf, _Max_: inf, _Vary_: True, _Fit_: True
    * __4__: _Value_: 0.00016, _Min_: -inf, _Max_: inf, _Vary_: True, _Fit_: True


## Datasets

### _dataset1_

* _Dataset Scaling_: None
* _Initial Concentration_: init
* _Megacomplexes_: ['mc1']
* _Megacomplex Scalings_: None
* _Irf_: irf

## Compartments

* s1
* s2
* s3
* s4

## Megacomplexes

### _mc1_
* _K-Matrices_: ['km1']


## K-Matrices

### _km1_

compartment |    __s1__ |    __s2__ |    __s3__ |    __s4__ |
------------|-----------|-----------|-----------|-----------|
     __s1__ |           |           |           |           |
     __s2__ | kinetic.1 |           |           |           |
     __s3__ |           | kinetic.2 |           |           |
     __s4__ |           |           | kinetic.3 | kinetic.4 |

## IRFs

### _irf_
* _Type_: 'Gaussian'
* _Center_: ['irf.center']
* _Width_: ['irf.width']
* _Center Dispersion_: []
* _Width Dispersion_: []
* _Scale_: []
* _Nomalize_: True



In [127]:
result = model.fit()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
display(Markdown(str(result)))

In [ ]:
## Plot the fitted data

fitted_dataset = result.fitted_data('dataset1')
data = fitted_dataset.get()

plt.figure(figsize=(20, 20))

# Full Residual
plt.subplot(4, 4, 1)
plt.title('Residual')
plt.pcolormesh(time_axis, spectral_axis, result.final_residual())

residual_svd = result.final_residual_svd()
lsvd = residual_svd[2] 
svals = residual_svd[1]
rsvd = residual_svd[0] 

# Plot left singular vectors (LSV, times, first 4)
plt.subplot(4, 4, 2)
plt.title('LSV Residual')
plt.plot(time_axis, lsvd[:, range(4)])

# Plot singular values (SV)
plt.subplot(4, 4, 3)
plt.title('SVals Residual')
n_svals = min(10, min(len(time_axis), len(spectral_axis)))
plt.plot(range(n_svals), svals[:n_svals], 'ro')
plt.yscale('log')

# Plot right singular vectors (RSV, wavelengths, first 4)
plt.subplot(4, 4, 4)
plt.title('RSV Residual')
plt.plot(spectral_axis, rsvd[:, range(4)])